# Import and get ready

In [1]:
!pip install autocorrect          
from autocorrect import Speller   # things we need for spell checking
check = Speller(lang='en')

import csv                        # csv is for importing and working with csv files

import datetime
date = datetime.date.today()

import nltk                       # get nltk 
from nltk import word_tokenize    # and some of its key functions
from nltk import sent_tokenize    
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.corpus import wordnet                    # Finally, things we need for lemmatising!
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
nltk.download('averaged_perceptron_tagger')        # Like a POS-tagger...
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('webtext')

import numpy as np
import os                         # os is a module for navigating your machine (e.g., file directories).
import pandas as pd
import statistics
import re                         # things we need for RegEx corrections

English_punctuation = "-!\"#$%&()'*+,./:;<=>?@[\]^_`{|}~''“”"      # Things for removing punctuation, stopwords and empty strings
table_punctuation = str.maketrans('','', English_punctuation)  

print("Succesfully imported necessary modules")    # The print statement is just a bit of encouragement!

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package webtext to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package webtext is already up-to-date!


Succesfully imported necessary modules


In [2]:
print(os.listdir("..\\for_analysis\\ESHG")) # This is how to see the contents of any folders shown in the last contents check
print(os.listdir("..\\counts\\ESHG")) # This is how to see the contents of any folders shown in the last contents check


['2001abstractICHG.txt', '2002Abstracts.txt', '2003Abstracts.txt', '2004Abstracts.txt', '2005Abstracts.txt', '2006Abstracts.txt', '2007Abstracts.txt', '2008Abstracts.txt', '2009Abstracts.txt', '2010Abstracts.txt', '2011Abstracts.txt', '2012Abstracts.txt', '2013Abstracts.txt', '2014Abstracts.txt', '2015Abstracts.txt', '2016Abstracts.txt', '2017 electronic posters.txt', '2017 oral presentations.txt', '2017 posters.txt', '2018 electronic posters.txt', '2018 EMPAG.txt', '2018 oral presentation.txt', '2018 posters.txt', '2019 oral presentation.txt', '2019 posters.txt', '2019 posters2.txt', '2020 eposters.txt', '2020 interactive eposter.txt', '2020 oral presentation.txt', '2021 eposters.txt', '2021 oral presentations.txt']
['select.csv', 'totals.csv']


# Read in and check useful things

In [3]:
totals_df = pd.read_csv('..\\counts\\ESHG\\totals.csv', header=None)
totals_df.columns = ["filename", "abstracts"]
totals_df

,filename,abstracts
0,2001abstractICHG,7805
1,2002Abstracts,5517
2,2003Abstracts,4539
3,2004Abstracts,13
4,2005Abstracts,7289
5,2006Abstracts,6026
6,2007Abstracts,7283
7,2008Abstracts,9794
8,2009Abstracts,7981
9,2010Abstracts,7795


In [142]:
raw_select_abstracts = []

with open('..\\counts\\ESHG\\select.csv', newline='', encoding = "ISO-8859-1") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        raw_select_abstracts.append(row)
        
select_abstracts = (list(filter(lambda x: x, raw_select_abstracts)))
    

In [5]:
select_df = pd.DataFrame.from_records(select_abstracts)
total_select = select_df.iloc[:,0].value_counts()


In [6]:
autism_count = select_df.iloc[:,0].value_counts().rename_axis('unique_values').to_frame('counts')

autism_count = autism_count.reset_index()
autism_count.columns = ['filename', 'autism_abstracts']

autism_count

,filename,autism_abstracts
0,2013Abstracts,137
1,2016Abstracts,132
2,2015Abstracts,129
3,2014Abstracts,116
4,2012Abstracts,108
5,2011Abstracts,104
6,2018 posters,98
7,2019 posters,92
8,2020 interactive eposter,85
9,2010Abstracts,74


In [7]:
merged_counts = pd.merge(totals_df, autism_count, on = 'filename')


In [8]:
merged_counts


,filename,abstracts,autism_abstracts
0,2001abstractICHG,7805,21
1,2002Abstracts,5517,33
2,2003Abstracts,4539,19
3,2005Abstracts,7289,32
4,2006Abstracts,6026,35
5,2007Abstracts,7283,41
6,2008Abstracts,9794,72
7,2009Abstracts,7981,42
8,2010Abstracts,7795,74
9,2011Abstracts,9133,104


In [12]:
merged_counts['year'] = merged_counts['filename'].str.extract(r'(\d{4})')
merged_counts


,filename,abstracts,autism_abstracts,year
0,2001abstractICHG,7805,21,2001
1,2002Abstracts,5517,33,2002
2,2003Abstracts,4539,19,2003
3,2005Abstracts,7289,32,2005
4,2006Abstracts,6026,35,2006
5,2007Abstracts,7283,41,2007
6,2008Abstracts,9794,72,2008
7,2009Abstracts,7981,42,2009
8,2010Abstracts,7795,74,2010
9,2011Abstracts,9133,104,2011


In [15]:
print(merged_counts['abstracts'].sum())
print(merged_counts['autism_abstracts'].sum())

167336
1694


In [17]:
print(merged_counts[['year', 'abstracts']].groupby('year').sum('abstracts'))
#print(merged_counts['autism_abstracts'].sum())

      abstracts
year           
2001       7805
2002       5517
2003       4539
2005       7289
2006       6026
2007       7283
2008       9794
2009       7981
2010       7795
2011       9133
2012       8777
2013      10626
2014       9041
2015       7978
2016      10734
2017      10405
2018      10429
2019       8250
2020       9590
2021       8344


# Working with the contents of the selected abstracts

##  Dump the content of the selected abtsracts into one big string

In [28]:
bag_of_abstracts = ""

for abstract in (select_abstracts):
    bag_of_abstracts += abstract[3]
        

## tokenize that big string, and transform the tokens into lowercase

In [19]:
abstract_token_word = word_tokenize(bag_of_abstracts)
abstract_token_word_lower = [word.lower() for word in abstract_token_word]

print(abstract_token_word_lower[:10]) 

['p08', '.10', 'rs4307059', 'and', 'rs4141463', 'study', 'in', 'an', 'italian', 'cohort']


## Remove the punctuation from the lowercased tokens

In [20]:
abstract_t_w_l_np = [w.translate(table_punctuation) for w in abstract_token_word_lower]  
                                                               # Iterate over corpus_words, turning punctuation to nothing.
print(abstract_t_w_l_np[:10])       

['p08', '10', 'rs4307059', 'and', 'rs4141463', 'study', 'in', 'an', 'italian', 'cohort']


In [21]:
abstract_t_w_l_np = (list(filter(lambda x: x, abstract_t_w_l_np)))
print(abstract_t_w_l_np[:10])

['p08', '10', 'rs4307059', 'and', 'rs4141463', 'study', 'in', 'an', 'italian', 'cohort']


## Remove stopwords

In [ ]:
print(sorted(stop_words))        # just an option to check what counts as a stopword if you want to see

In [22]:
abstract_t_w_l_np_nsw = []

for word in abstract_t_w_l_np:
    if word not in stop_words:
        abstract_t_w_l_np_nsw.append(word)
        
        
print(abstract_t_w_l_np_nsw[:10])

['p08', '10', 'rs4307059', 'rs4141463', 'study', 'italian', 'cohort', 'neck', 'surgery', 'university']


## Spell check the lowercased "real word" tokens

In [ ]:
len(abstract_t_w_l_np_nsw)

In [ ]:
# runs forever without finishing. Find alternative or skip it?

abstract_t_w_l_np_cs = []

for word in abstract_t_w_l_np_nsw:
    abstract_t_w_l_np_cs.append(check(word))    

print(abstract_t_w_l_np_cs[:10])

In [23]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()
abstract_t_w_l_np_nsw_stem = [porter.stem(word) for word in abstract_t_w_l_np_nsw]
print(abstract_t_w_l_np_nsw_stem[:10])

['p08', '10', 'rs4307059', 'rs4141463', 'studi', 'italian', 'cohort', 'neck', 'surgeri', 'univers']


In [24]:
from collections import Counter
counts = Counter(abstract_t_w_l_np_nsw_stem)
print(type(counts))

<class 'collections.Counter'>


In [25]:
print(counts.most_common(100))

[('gene', 3535), ('none', 3535), ('patient', 3516), ('genet', 3106), ('de', 2423), ('univers', 2210), ('disord', 2201), ('autism', 2057), ('mutat', 1878), ('asd', 1736), ('clinic', 1705), ('studi', 1582), ('syndrom', 1560), ('delet', 1557), ('associ', 1535), ('variant', 1439), ('case', 1427), ('chromosom', 1398), ('hospit', 1252), ('use', 1240), ('medic', 1236), ('phenotyp', 1228), ('analysi', 1171), ('result', 1163), ('report', 1160), ('unit', 1126), ('c', 1115), ('famili', 1099), ('j', 1091), ('e', 1033), ('genom', 1025), ('two', 983), ('franc', 978), ('spectrum', 973), ('p', 958), ('includ', 958), ('region', 953), ('present', 947), ('sequenc', 934), ('diseas', 929), ('featur', 921), ('cnv', 906), ('identifi', 881), ('caus', 872), ('sever', 854), ('show', 804), ('delay', 794), ('disabl', 784), ('human', 770), ('itali', 762), ('institut', 756), ('one', 752), ('l', 731), ('medicin', 725), ('2', 720), ('intellectu', 713), ('r', 712), ('molecular', 707), ('duplic', 694), ('cell', 687), (

## Person-first and identity first

In [40]:
!pip install spacy
import re 
import string 
import nltk 
import spacy 
import pandas as pd 
import numpy as np 
import math 
from tqdm import tqdm 

from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 

pd.set_option('display.max_colwidth', 200)

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\python39\\Scripts\\spacy.exe' -> 'c:\\python39\\Scripts\\spacy.exe.deleteme'



  Using cached spacy-3.4.1-cp39-cp39-win_amd64.whl (11.8 MB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)


In [45]:
!pip install spacy -q
import spacy
!python -m spacy download en_core_web_lg -q
from nltk.corpus import webtext

nlp = spacy.load('en_core_web_lg')

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [41]:
person_identity = nltk.sent_tokenize(bag_of_abstracts)


In [42]:
type(person_identity)

list

In [111]:
person =""

for sentence in person_identity:
    if any(s in sentence for s in ['autisic', 'Autisic', 'autism', 'Autism']):
        person += sentence


In [112]:
len(person)

631947

In [113]:
for tok in person_doc: 
  print(tok.text, "-->",tok.dep_,"-->", tok.pos_)

Patients --> nsubj --> NOUN
4 --> nummod --> NUM
and --> cc --> CCONJ
5 --> conj --> NUM
presented --> ROOT --> VERB
with --> prep --> ADP
autistic --> amod --> ADJ
features --> pobj --> NOUN
. --> punct --> PUNCT
Primary --> amod --> ADJ
microcephalies --> ROOT --> NOUN
( --> punct --> PUNCT
MicroCephaly --> compound --> PROPN
Primary --> compound --> PROPN
Hereditary --> appos --> PROPN
, --> punct --> PUNCT
MCPH --> appos --> PROPN
) --> punct --> PUNCT
We --> nsubj --> PRON
report --> ROOT --> VERB
two --> nummod --> NUM
children --> dobj --> NOUN
with --> prep --> ADP
autistic --> amod --> ADJ
behavior --> pobj --> NOUN
, --> punct --> PUNCT
mental --> amod --> ADJ
retardation --> conj --> NOUN
and --> cc --> CCONJ
have --> aux --> AUX
often --> advmod --> ADV
been --> auxpass --> AUX
presented --> conj --> VERB
as --> prep --> ADP
a --> det --> DET
developmental --> amod --> ADJ
disorders --> pobj --> NOUN
resulting --> acl --> VERB
in --> prep --> ADP
dysmorphic --> amod --> ADJ

children --> pobj --> NOUN
in --> prep --> ADP
western --> amod --> ADJ
saudi --> compound --> PROPN
arabia --> compound --> PROPN
autism --> compound --> NOUN
spectrum --> compound --> NOUN
disorders --> pobj --> NOUN
( --> punct --> PUNCT
ASD).To --> nsubj --> PROPN
identify --> parataxis --> VERB
these --> det --> DET
likely --> amod --> ADJ
functional --> amod --> ADJ
M. --> compound --> PROPN
M. --> compound --> PROPN
Alwasiyah1 --> dobj --> PROPN
, --> punct --> PUNCT
C. --> compound --> PROPN
Trujillo2 --> appos --> NOUN
; --> punct --> PUNCT
elements --> appos --> NOUN
, --> punct --> PUNCT
we --> nsubj --> PRON
combined --> parataxis --> VERB
three --> dobj --> NUM
previously --> advmod --> ADV
published --> acl --> VERB
large --> amod --> ADJ
cohorts --> dobj --> NOUN
1Aziziah --> compound --> PROPN
Maternity --> appos --> PROPN
& --> cc --> CCONJ
Children --> compound --> PROPN
Hospital --> conj --> PROPN
, --> punct --> PUNCT
Jeddah --> conj --> PROPN
, --> punct --> PUNCT


Although --> mark --> SCONJ
the --> det --> DET
molecular --> amod --> ADJ
basis --> nsubj --> NOUN
of --> prep --> ADP
this --> det --> DET
disorder --> pobj --> NOUN
has --> aux --> AUX
not --> neg --> PART
been --> advcl --> AUX
totally --> advmod --> ADV
dchromosomal --> acomp --> ADJ
abberations.(Oxford --> compound --> PROPN
Gene --> compound --> PROPN
Technology --> dobj --> PROPN
, --> punct --> PUNCT
Oxford --> nmod --> PROPN
, --> punct --> PUNCT
UK --> nmod --> PROPN
) --> punct --> PUNCT
microchips --> appos --> NOUN
and --> cc --> CCONJ
scanned --> conj --> VERB
with --> prep --> ADP
Agilent --> compound --> PROPN
Microarray --> compound --> PROPN
Scanner --> pobj --> NOUN
( --> punct --> PUNCT
Agilent --> compound --> PROPN
Technologies --> nsubjpass --> PROPN
, --> punct --> PUNCT
Palo --> compound --> PROPN
Alto --> npadvmod --> PROPN
, --> punct --> PUNCT
CAP1 --> compound --> PROPN
  --> dep --> SPACE
autism --> compound --> NOUN
spectrum --> compound --> NOUN
disorde

p>0.05 --> appos --> PROPN
) --> punct --> PUNCT
, --> punct --> PUNCT
there --> expl --> PRON
was --> ccomp --> VERB
a --> det --> DET
between --> prep --> ADP
serum --> compound --> NOUN
visfatin --> compound --> NOUN
levels --> pobj --> NOUN
and --> cc --> CCONJ
the --> det --> DET
serum --> compound --> NOUN
insulin --> conj --> NOUN
, --> punct --> PUNCT
TNF --> conj --> PROPN
- --> punct --> PUNCT
Î± --> dep --> SPACE
, --> punct --> PUNCT
BMI --> conj --> PROPN
, --> punct --> PUNCT
significant --> amod --> ADJ
activity --> compound --> NOUN
increase --> appos --> NOUN
in --> prep --> ADP
the --> det --> DET
autistic --> amod --> ADJ
groups --> pobj --> NOUN
with --> prep --> ADP
R --> compound --> PROPN
allele --> compound --> NOUN
number --> pobj --> NOUN
HbA1c --> attr --> PROPN
, --> punct --> PUNCT
insulin --> compound --> NOUN
resistance --> conj --> NOUN
, --> punct --> PUNCT
proteinuria --> conj --> NOUN
and --> cc --> CCONJ
creatinine --> compound --> NOUN
clearence -->

BarreÃ±a --> appos --> PROPN
; --> punct --> PUNCT
United --> compound --> PROPN
States --> appos --> PROPN
. --> punct --> PUNCT
Several --> amod --> ADJ
from --> prep --> ADP
a --> det --> DET
promoter --> pobj --> NOUN
located --> acl --> VERB
upstream --> advmod --> NOUN
of --> prep --> ADP
exon --> pobj --> NOUN
1 --> nummod --> NUM
, --> punct --> PUNCT
while --> mark --> SCONJ
the --> det --> DET
Î² --> compound --> PROPN
neurexins --> nsubj --> NOUN
are --> advcl --> AUX
tran- --> dep --> VERB
recent --> amod --> ADJ
studies --> nsubj --> NOUN
have --> aux --> AUX
implicated --> ROOT --> VERB
NL --> dobj --> PROPN
in --> prep --> ADP
autistic --> amod --> ADJ
disorders --> pobj --> NOUN
and --> cc --> CCONJ
mental --> amod --> ADJ
retar- --> conj --> NOUN
scribed --> acl --> VERB
from --> prep --> ADP
an --> det --> DET
intragenic --> amod --> ADJ
downstream --> amod --> ADJ
promoter --> pobj --> NOUN
. --> punct --> PUNCT
He --> nsubj --> PRON
had --> ROOT --> VERB
some --> de

...

Rassu2 --> appos --> PROPN
, --> punct --> PUNCT
L. --> compound --> PROPN
Boccone3 --> conj --> PROPN
, --> punct --> PUNCT
D. --> compound --> PROPN
Gasperini3 --> appos --> PROPN
, --> punct --> PUNCT
A. --> compound --> PROPN
Cao1 --> appos --> PROPN
, --> punct --> PUNCT
L. --> compound --> PROPN
Crisponi1 --> conj --> PROPN
; --> punct --> PUNCT
and --> cc --> CCONJ
post --> amod --> ADJ
- --> amod --> ADJ
meiotic --> amod --> ADJ
cells --> conj --> NOUN
from --> prep --> ADP
cryo --> compound --> NOUN
- --> punct --> PUNCT
sections --> pobj --> NOUN
of --> prep --> ADP
Rhesus --> compound --> PROPN
macaque --> pobj --> NOUN
testis --> npadvmod --> PROPN
1Istituto --> npadvmod --> PROPN
di --> ROOT --> PROPN
Neurogenetica --> compound --> PROPN
e --> compound --> PROPN
Neurofarmacologia --> compound --> PROPN
INN --> compound --> PROPN
- --> punct --> PUNCT
CNR --> nsubj --> PROPN
, --> punct --> PUNCT
Monserrato --> appos --> PROPN
( --> punct --> PUNCT
CA --> appos --> PROPN
) 

syndrome --> pobj --> NOUN
, --> punct --> PUNCT
Heller --> compound --> PROPN
syn- --> compound --> NOUN
anomaly --> appos --> NOUN
. --> punct --> PUNCT
Whole --> amod --> ADJ
exome --> compound --> NOUN
sequencing --> ROOT --> NOUN
of --> prep --> ADP
ABSTRACTS --> compound --> NOUN
POSTERS --> pobj --> NOUN
Back --> advmod --> ADV
to --> prep --> ADP
index --> compound --> NOUN
gDNA --> pobj --> NOUN
from --> prep --> ADP
the --> det --> DET
boy --> pobj --> NOUN
and --> cc --> CCONJ
his --> poss --> PRON
parents --> nsubj --> NOUN
identified --> conj --> VERB
a --> det --> DET
de --> nmod --> X
novo --> nmod --> PROPN
, --> punct --> PUNCT
novel --> amod --> ADJ
variant --> dobj --> NOUN
in --> prep --> ADP
250 --> nummod --> NUM
K --> compound --> PROPN
Nsp --> compound --> PROPN
chips --> pobj --> NOUN
, --> punct --> PUNCT
exome --> compound --> NOUN
capture --> dobj --> NOUN
using --> acl --> VERB
NimbleGen --> dobj --> PROPN
SeqCap --> compound --> PROPN
EZ --> compound --> P

895 --> appos --> NUM
from --> prep --> ADP
Germany --> pobj --> PROPN
, --> punct --> PUNCT
419 --> appos --> NUM
from --> prep --> ADP
Norway --> pobj --> PROPN
) --> punct --> PUNCT
and --> cc --> CCONJ
2700 --> nummod --> NUM
controls --> nsubj --> VERB
nical --> amod --> ADJ
assessment --> dobj --> NOUN
of --> prep --> ADP
all --> det --> DET
autistic --> amod --> ADJ
and --> cc --> CCONJ
cognitively --> advmod --> ADV
impaired --> conj --> ADJ
children --> pobj --> NOUN
, --> punct --> PUNCT
as --> advmod --> ADV
well --> amod --> ADV
( --> punct --> PUNCT
2366 --> nummod --> NUM
/ --> punct --> SYM
339).She --> appos --> NOUN
subsequently --> advmod --> ADV
showed --> conj --> VERB
nose --> dobj --> NOUN
, --> punct --> PUNCT
small --> amod --> ADJ
mouth --> appos --> NOUN
with --> prep --> ADP
high --> amod --> ADJ
arched --> amod --> ADJ
palate --> amod --> NOUN
, --> punct --> PUNCT
crowded --> amod --> ADJ
teeth --> pobj --> NOUN
, --> punct --> PUNCT
bilateral --> amod --> 

CACNA1A --> compound --> PROPN
gene --> nmod --> NOUN
tedsynaptic --> amod --> ADJ
dysfunction --> pobj --> NOUN
, --> punct --> PUNCT
as --> prep --> ADP
a --> det --> DET
key --> amod --> ADJ
mechanism --> pobj --> NOUN
in --> prep --> ADP
the --> det --> DET
patho- --> compound --> NOUN
NM_001127222.1 --> pobj --> NOUN
: --> punct --> PUNCT
c.2259_2260insCG;(p --> dep --> X
. --> punct --> PUNCT
A754Rfs*6)and --> cc --> CCONJ
physiology --> nsubj --> NOUN
of --> prep --> ADP
ASD.There --> pobj --> ADV
are --> ROOT --> AUX
an --> det --> DET
autistic --> amod --> ADJ
- --> punct --> PUNCT
like --> amod --> ADJ
Hospitalario --> compound --> PROPN
Universitario --> compound --> PROPN
de --> compound --> PROPN
Albacete --> attr --> PROPN
. --> punct --> PUNCT
, --> punct --> PUNCT
Albacete --> npadvmod --> PROPN
, --> punct --> PUNCT
Spain --> conj --> PROPN
, --> punct --> PUNCT
2NeurologÃ­a --> nummod --> NUM
manner --> conj --> NOUN
and --> cc --> CCONJ
phases --> conj --> NOUN
of --

feeding --> amod --> VERB
difï¬culties --> conj --> NOUN
, --> punct --> PUNCT
autistic --> amod --> ADJ
behavior --> conj --> NOUN
, --> punct --> PUNCT
hearing --> acl --> VERB
oraspartofasyndrome --> dobj --> PROPN
. --> punct --> PUNCT
FGFR2,FGFR3,FGFR1,TWIST1 --> ROOT --> NUM
impairment --> nmod --> NOUN
, --> punct --> PUNCT
short --> amod --> ADJ
stature --> nsubj --> NOUN
, --> punct --> PUNCT
microcephaly --> nmod --> NOUN
, --> punct --> PUNCT
and --> cc --> CCONJ
facial --> amod --> ADJ
dys- --> nmod --> NOUN
and --> cc --> CCONJ
EFNB --> conj --> PROPN
genes --> appos --> NOUN
are --> ROOT --> AUX
major --> amod --> ADJ
causative --> amod --> ADJ
genes --> attr --> NOUN
of --> prep --> ADP
genetic --> amod --> ADJ
morphisms --> pobj --> NOUN
[ --> punct --> PUNCT
Menke --> nmod --> PROPN
et --> nmod --> X
al --> dep --> PROPN
. --> appos --> PROPN
, --> punct --> PUNCT
2016 --> appos --> NUM
; --> punct --> PUNCT
Menke --> compound --> PROPN
et --> compound --> X
al --> np

In [131]:
pattern_1 = [{"TEXT": {"REGEX": "^[Aa]utistic$"}},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {"POS": "NOUN"}]

In [132]:
# Matcher class object 
matcher = Matcher(nlp.vocab) 
matcher.add("matching_1", [pattern_1]) 

autistic =[]

matches = matcher(person_doc) 
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = person_doc[start:end]  # The matched span
    autistic.append(span.text)

In [159]:
autistic_no_dups = list(set(autistic))
with open('..\\counts\\ESHG\\autism.csv', "w", encoding='ISO-8859-1') as outfile:
        write = csv.writer(outfile)
        for item in autistic_no_dups:
            write.writerow([item])



In [157]:
for item in autistic_no_dups:
    print(item)

autistic features
autistic behaviour.individuals
autistic boy
autistic mole-
autistic Basque sample
autistic traits
autistic behaviuor
autistic behaviour.delayandbehavioraldisorder/ASDtraits,associatedwith
autistic dis- ing
autistic regression
autistic patientâs
autistic phenotype
autistic variants
autistic Results
autistic trait
autistic features,358 aggressiveness
autistic behaviour
autistic behaviour.8,9
autistic subjects
autistic variome
autistic image
autistic disorders
autistic population
autistic gene
autistic spectrum
autistic syndrome
autistic ethnic populations
autistic girl
autistic traits.nal features
autistic behavior
autistic populations
autistic indi- variability
autistic patientsâ
autistic group
autistic microcephaly
autistic brain
autistic patients
autistic child-
autistic cases
autistic symptoms
autistic patient
autistic syndromes
autistic beha- disorder
autistic groups
autistic behaviour.1University
autistic unrelated patients
autistic features.feeding
autistic v

In [138]:
pattern_2 = [{"POS": "NOUN"},
             {'LOWER': 'with'},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {"TEXT": {"REGEX": "^[Aa]utism$"}}]

In [139]:
# Matcher class object 
matcher = Matcher(nlp.vocab) 
matcher.add("matching_1", [pattern_2]) 

person_with =[]
matches = matcher(person_doc) 
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = person_doc[start:end]  # The matched span
    person_with.append(span.text)

In [160]:
person_with_no_dups = list(set(person_with))
with open('..\\counts\\ESHG\\person_with.csv', "w", encoding='ISO-8859-1') as outfile:
        write = csv.writer(outfile)
        for item in person_with_no_dups:
            write.writerow([item])

